# QUESTIONNAIRE ANALYSIS AUTOMATION

## Import Libraries 

In [ ]:
import pandas as pd # library for data analysis and data manipulation
import numpy as np # library to work with arrays
import re # library for data cleaning
from IPython.display import HTML # display a pandas dataframe to HTML
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import warnings
from statistics import mean
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

## Read the dataset

In [ ]:
df= pd.read_excel(r'responses.xlsx') #import dataset as pandas dataframe
print('Dataset dimensions:',df.shape)

## Find the columns with the same name and replace values to be equal to column names
In some columns we have exactly the same name. This happens because we may ask similar questions that have same answers. 
Let say that we have the option "brandA" in to multiple questions. Then, the corresponding columns will interpret by Python like "BrandA","BrandA.1","BrandA.2" etc. But the answers will continue to be "BrandA" for all the questions.
To run the code succesfully, we want all of them to have the same name.

**Just run it without interaction!**

In [ ]:
# Find the columns with the same name
new_list = [item.split(".")[0] for item in list(df.columns)]

counted_items = Counter(new_list)
duplicates = []

for item, count in counted_items.items():
    if count > 1:
        duplicates.append(item)
print(duplicates)

# Change the values of duplicates columns
for col in df.columns:
    for i in duplicates:
        if i in col:
            col_dict = {val: col for val in df[col].dropna().unique()}
            df[col] = df[col].replace(col_dict)

## Age coding
It is very common that we want to group the age. You can adjust the ranges according your needs.

In [ ]:
def rename_values(val):
    if val <= 30:
         return '18-30'
    elif val >= 31 and val <= 40:
        return '31-40'
    elif val >= 41 and val <= 50:
        return '41-50'
    else:
        return '>50'

# use the apply method to rename the values
df['Ηλικία;'] = df['Ηλικία;'].apply(rename_values)

## Dictionary

We create a dictionary of our spreadsheet and that's because it is more efficient to write every time the index of the column we want to analyze instead of the whole question.

**Just run it!**

In [ ]:
#create 2 empty lists
list1=[]
list2=[]

col=list(df.columns) #create a list with all the column names

for i in col: # columns names
    list1.append(i) # create a list with the column names
    list2.append(col.index(i)) # create a list with indexes
       
res = dict(zip(list1,list2)) #μετατρέπει 2 λίστες σε λεξικό
res

## Specific orders 

As mentioned above, for the ordered questions we need to define the desired order we want to appear on the diagrams, tables.
That's because this way we don't need to change them manually in every table!

**Here there some modifations are needed every time you run the code!**

The number (65,63 etc) indicates the index of the column that has a certain order. Next in the brackets you should write the right order.
Here also you should define the questions that may have not certain order but you need to analyze with multiple questions.
You write them like this: 

list(df.iloc[:, 66].value_counts().index)

In [ ]:
specific = {
    65: ["18-24", "25-30"],  # Age
    63: ['Άντρας', 'Γυναίκα'],  # Gender
    66: list(df.iloc[:, 66].value_counts().index),  # Identity
}
# Create a dictionary with specific orders and their indexes
lexico = specific.copy()
lexico

## Function for both ordered and no ordered simple questions

**Just run it!**


You call the function like this: func(input1,input2,input3)
- **input 1:** main question: which question do you want to analyze?
- **input 2:** define the specific order (if exists) as you defined it in the lexico above. If not, just enter "".
- **input 3:** define the dataframe. May it is the initial may you should use a subset. It depends on your analysis needs.

For example let say you want to analyze the question with index 1 that has as specific order "days" and you want crosstabs with the questions with index 10 and 15 and let say that the dataframe is called df.

So you call the function: func(1,days,df) and then in the box appearing you enter 10 15.

Then, after you call the function, it will be asked you to enter the indexes of the columns with which you want crosstabs.
You can enter as many you want with a space between.
Then the desired frequencies and crosstabs will appear!

In [ ]:
def func(main_question,order_of_main_question,new_df):
    
    
    indexes_of_columns= input("Enter the indexes of the interested columns (separated by space) : ")
    indexes_of_columns=indexes_of_columns.split() 
    indexes_of_columns = [eval(i) for i in indexes_of_columns]
    
    df=new_df
    lista=[] # store all the pandas dataframes (frequencies and crosstabs)
    
        
    if main_question in range(len(res)):
        # frequency 
        freq=pd.DataFrame(df.iloc[:,main_question].value_counts(normalize=True).round(2)*100)
        freq=freq.astype(str).applymap(lambda x:x + '%')
        
        if order_of_main_question in specific :
                display(freq.reindex(order_of_main_question))
        else:
                display(freq)
     
        
        # crosstabs
        for i in indexes_of_columns: 
                if i in list((lexico.keys())):
                    crosst0=pd.crosstab(df.iloc[:,main_question],df.iloc[:,i],normalize='columns').reindex(columns=lexico[i]) 
                else:
                    crosst0=pd.crosstab(df.iloc[:,main_question],df.iloc[:,i],normalize='columns')       
                   
                for col in range(len(crosst0.columns)):
                        crosst0.iloc[:,col]= np.round(crosst0.iloc[:,col],decimals = 2)*100  
                        
                lista.append(crosst0)
                
        for j in range(len(lista)):
            if order_of_main_question in specific :
                display(lista[j].reindex(index=list(order_of_main_question)))
            else:
                display(lista[j].reindex(index=list(freq.index.values)))

              
         # return the tables       
    return 

## Function for both ordered and no ordered multiple questions

**Just run it!**

When a question has multiple choices, each choice is a column. So:
You call the function like this: multiple(input1,input2,input3)
- **input 1:** the index of first column (first choice of the corresponding question - you find it on above lexico)
- **input 2:** the index of second column + 1 ! (last choice of the corresponding question - you find it on above lexico (+1 because python counts from zero)
- **input 3:** define the dataframe. May it is the initial may you should use a subset. It depends on your analysis needs.

For example let say you want to analyze the multiple question that its answers are in the columns 10 until 15 and you want crosstabs with the questions with index 10 and 15 and let say that the dataframe is called df.

So you call the function: multiple(10,16,df) and then in the box appearing you enter 10 15.

Then, after you call the function, it will be asked you to enter the indexes of the columns with which you want crosstabs.
You can enter as many you want with a space between.
Then the desired frequencies and crosstabs will appear!

In [ ]:
def multiple(i,j,df):
    
    
    indexes_of_columns= input("Enter the indexes questions for crosstabs (separated by space) : ")
    indexes_of_columns=indexes_of_columns.split() # convert the input into a list
    indexes_of_columns = [eval(i) for i in indexes_of_columns] # strings into integers
    
    
    lista=[]
    
    # Frequencies
    
    new_df=df 
    new_df=new_df.iloc[:, i:j]  
    new_df=new_df.dropna(axis = 0, how = 'all') 
    
    variables = new_df.columns 
    
    frequency = [] 
    names = [] 
    percentage = []
     
    for i in variables:    
        frequency.append(sum(1 if (x == i) else 0  for x in new_df[i])) #finds the frequencies
        names.append(i)
    
    for i in frequency:
        percentage.append(round(i/new_df.shape[0],2)* 100) # finds the valid percentages
     
    a=pd.DataFrame(list(zip(names,percentage)),columns=['','Frequency'],index=None) #pandas dataframe with frequencies
    a=a.sort_values('Frequency',ascending=False)
    display(HTML(a.to_html(index=False)))
    
    # Crosstabs
    
    for i in indexes_of_columns : 

        independant_cols=list(a.iloc[:,0])  
        cols  = [df.iloc[:,i].name] + list(a.iloc[:,0]) 
        groupings = (df[cols].groupby(df.iloc[:,i]).count())
    
        for x in list(a.iloc[:,0]) :
            groupings['{} '.format(x)] = round(groupings[x]/groupings.iloc[:,0]*100,0)
            groupings = groupings.drop(columns = [x])
            groupings  
            g1=groupings.reindex(lexico[i])     
       
        lista.append(g1)
    
               
    for j in range(len(lista)):
        display(lista[j].T)
              
       
    return 

## Tip: 
Save the file in a folder and make the adjustments. Then open a new ipynb file in the same folder and in the first cell run:

**%run [name_of_current_file]ipynb # run an other Jupyter Notebook**

This way you can have only the outputs in your file to be more tidy.